### 📘 사이킷런 군집(Clustering) 수업자료

---

### 1️⃣ 군집(Clustering)이란?

군집은 **정답(label) 없이 비슷한 데이터끼리 묶는 비지도학습 방법**이다.

- 지도학습 → 정답 있음
- 비지도학습 → 정답 없음
- 목표 → 데이터의 숨겨진 구조 찾기

예시:
- 고객 세분화
- 뉴스 기사 주제 분류
- 이상치 탐지
- 이미지 그룹화

---

### 2️⃣ 대표적인 군집 알고리즘

### 🔵 1. K-Means

### 특징
- 가장 많이 사용
- 중심점(centroid) 기반
- k 값을 미리 정해야 함

### 동작 과정
1. 중심점 k개 생성
2. 가까운 중심으로 데이터 할당
3. 중심 재계산
4. 반복

---

### 🔵 2. DBSCAN

### 특징
- 밀도 기반 알고리즘
- k값 필요 없음
- 이상치 탐지 가능
- 불규칙한 모양의 군집 가능

### 주요 파라미터
- eps: 반경 거리
- min_samples: 최소 데이터 개수

---

### 🔵 3. 계층적 군집 (Hierarchical Clustering)

### 특징
- 트리 구조로 군집 생성
- 덴드로그램으로 시각화
- 군집 개수를 나중에 결정 가능

---

### 3️⃣ 실습: 데이터 생성

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=3, random_state=42)

plt.scatter(X[:,0], X[:,1])
plt.title("Original Data")
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. 랜덤 시드 고정 (항상 같은 결과가 나오도록)
np.random.seed(42)

# 2. 강남구 주요 거점 좌표 설정 (중심점 역할)
# (현실성을 위해 실제 좌표 기반으로 설정했습니다)
# [위도(Lat), 경도(Lon)]
centers = {
    'Gangnam_Stn': [37.4979, 127.0276],  # 강남역 (상업지구)
    'Samsung_COEX': [37.5118, 127.0593], # 삼성역 (오피스/쇼핑)
    'Dogok_Daechi': [37.4896, 127.0543], # 도곡/대치 (주거/학원가)
    'Apgujeong': [37.5278, 127.0378],    # 압구정 (고급 주거/상권)
    'Suseo_Stn': [37.4874, 127.1013]     # 수서역 (교통 요충지)
}

# 3. 데이터 생성 함수
def generate_gangnam_data(n_samples=1000):
    data = []
    
    # 각 거점별로 할당할 데이터 개수 (불균형하게 설정하여 현실감 부여)
    # 강남역: 300명, 삼성역: 200명, 대치: 250명, 압구정: 150명, 수서: 100명
    counts = [300, 200, 250, 150, 100]
    
    # 거점별 퍼짐 정도 (Standard Deviation)
    # 0.005 도는 약 반경 500m~600m 정도입니다.
    spreads = [0.006, 0.005, 0.005, 0.004, 0.007] 
    
    keys = list(centers.keys())
    
    for i, key in enumerate(keys):
        center = centers[key]
        count = counts[i]
        spread = spreads[i]
        
        # 정규분포(Gaussian)를 이용해 중심지 근처에 점들을 찍음
        latitudes = np.random.normal(center[0], spread, count)
        longitudes = np.random.normal(center[1], spread, count)
        
        for lat, lon in zip(latitudes, longitudes):
            data.append([lat, lon])
            
    return np.array(data)

# 4. 데이터 생성 실행
X = generate_gangnam_data()

# 5. DataFrame으로 변환 (보기 좋게)
df_gangnam = pd.DataFrame(X, columns=['Latitude', 'Longitude'])

# 데이터 미리보기 (상위 5개)
print("생성된 강남구 고객 위치 데이터 예시:")
print(df_gangnam.head())

# 6. 시각화 (제대로 만들어졌는지 확인)
plt.figure(figsize=(10, 8))
plt.scatter(df_gangnam['Longitude'], df_gangnam['Latitude'], s=10, alpha=0.5, c='blue')
plt.title('Distribution of Customers in Gangnam-gu (Simulation)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)

# 주요 거점 표시 (빨간 X)
for key, coords in centers.items():
    plt.text(coords[1], coords[0], key, fontsize=12, fontweight='bold', color='red')
    plt.scatter(coords[1], coords[0], marker='x', c='red', s=100)

plt.show()

df_gangnam.to_csv('gangnam_starbuks.csv')

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns # 시각화를 예쁘게 하기 위해 추가

# 데이터 프레임 생성
data = pd.read_csv('gangnam_starbuks.csv')
df = pd.DataFrame(data, columns=['Latitude', 'Longitude'])
print(df)

# ==========================================
# 2. K-Means 알고리즘 적용 (핵심 부분)
# ==========================================

# K=3 (매장 3개를 짓겠다고 가정)
kmeans = KMeans(n_clusters=3, random_state=42)

# 학습 실행 (위도, 경도 데이터를 넣음)
kmeans.fit(df[['Latitude', 'Longitude']])

# 결과 1: 각 고객이 어느 매장(그룹)에 속하는지 라벨링
df['Cluster'] = kmeans.labels_

# 결과 2: 최적의 입지 3곳 좌표 (Centroids) 추출
optimal_locations = kmeans.cluster_centers_

# ==========================================
# 3. 결과 출력 및 시각화
# ==========================================

print(">>> [분석 결과] 최적의 입지 후보 3곳 (위도, 경도) <<<")
for i, loc in enumerate(optimal_locations):
    print(f"매장 {i+1} 후보지: 위도 {loc[0]:.6f}, 경도 {loc[1]:.6f}")

# 시각화 (산점도)
plt.figure(figsize=(12, 10))

# 1) 고객 데이터 그리기 (그룹별로 색깔 다르게)
sns.scatterplot(x='Longitude', y='Latitude', hue='Cluster', data=df, 
                palette='viridis', s=50, alpha=0.6, legend='full')

# 2) 최적 입지(중심점) 그리기 (빨간색 별표)
plt.scatter(optimal_locations[:, 1], optimal_locations[:, 0], 
            s=300, c='red', marker='*', label='Optimal Locations (Centroids)', edgecolors='black')

plt.title('Optimal Store Locations in Gangnam-gu (K-Means Clustering)', fontsize=15)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ✅ DBSCAN 실전 예시: "쇼핑몰 고객 이상행동(이상치) 탐지"
# - 목적: 고객을 자연스럽게 군집화하고, 'noise(-1)'로 찍히는 고객을 이상치로 찾는다.
# - 예: VIP/일반/가성비/충동구매 등 군집 + 사기/비정상 행동 패턴 탐지

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('out_visit.csv',index_col='Unnamed: 0')

print(df.head())

# ----------------------------
# 2) 스케일링(필수!)
# ----------------------------
# DBSCAN은 거리 기반이라 단위 차이가 크면 결과가 망가질 수 있음.
scaler = StandardScaler()
X = scaler.fit_transform(df[["monthly_spend", "visit_count"]])

# ----------------------------
# 3) DBSCAN 학습
# ----------------------------
# eps: 이웃으로 볼 반경
# min_samples: 이웃 최소 개수(코어 포인트 기준)
dbscan = DBSCAN(eps=0.35, min_samples=8)
labels = dbscan.fit_predict(X)

df["cluster"] = labels

# ----------------------------
# 4) 결과 확인
# ----------------------------
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = (labels == -1).sum()

print(f"클러스터 개수(노이즈 제외): {n_clusters}")
print(f"노이즈(이상치) 개수: {n_noise}")

# 군집별 개수
print("\n[군집별 고객 수]")
print(df["cluster"].value_counts().sort_index())

# ----------------------------
# 5) 시각화
# ----------------------------
plt.figure(figsize=(7, 5))
plt.scatter(df["monthly_spend"], df["visit_count"], c=df["cluster"])
plt.xlabel("Monthly Spend")
plt.ylabel("Visit Count")
plt.title("DBSCAN: Customer Clusters & Outliers (cluster=-1 is noise)")
plt.show()

# ----------------------------
# 6) 이상치(노이즈)만 뽑기
# ----------------------------
outliers = df[df["cluster"] == -1].copy()
print("\n[이상치(노이즈) 고객 샘플]")
print(outliers.sort_values(["monthly_spend", "visit_count"], ascending=False).head(20))
